# Resampling Options

:::{note}
Most of the options below require the [Datashader](https://datashader.org) library to be installed. The [`downsample`](option-downsample) option should preferably used with the [tsdownsample](https://github.com/predict-idlab/tsdownsample) library installed.
:::

:::{seealso}
To dive deeper into the Datashader-related option, users are encouraged to read HoloViews' [Large Data user guide](https://holoviews.org/user_guide/Large_Data.html) and Dashader's [website](https://datashader.org), in particular the [Plotting pitfalls user guide](https://datashader.org/user_guide/Plotting_Pitfalls.html).
:::

:::{warning}
Most of the examples below require to be run in a Jupyter notebook to experience their full interactivity (dynamic resampling after zoomin/panning).
:::

The `hvsampledata.synthetic_clusters` dataset is many examples below.

In [ ]:
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")
print(f"This dataset contains {len(df)} rows.")
print("Sample from each of the 5 clusters")
df.iloc[[int(i *len(df)) / 5 for i in range(5)]]

```{eval-rst}
.. plotting-options-table:: Resampling Options
```

(option-aggregator)=
## `aggregator`

The `aggregator` option determines how data is reduced when applying [`datashade=True`](option-datashade) or [`rasterize=True`](option-rasterize). Precisely, aggregators dynamically compute a single value per pixel when converting many raw data points into a rasterized image. By default, a simple `count` aggregator is applied, which creates a plot that displays how many data points are contained within each pixel.

This can be particularly useful for large datasets where plotting every point is impractical. Choosing the right aggregator helps highlight the relevant aspects of the data — whether you want to count point density, show average values, or observe minimums and maximums across regions.

Two main types of aggregators are available:
- Mathematical combination of data such as the `count` of data points per pixel or the `mean` of a dimension of the supplied dataset, including: `'any'`, `'count'`, `'mode'`, `'mean'`, `'sum'`, `'var'`, `'std'`.
- Selection of data from a dimension of the supplied dataset, or the index of the corresponding row in the dataset, including: `'first'`, `'last'`, `'min'`, `'max'`.

`aggregator` accepts either:
- A [Datashader reduction object](https://datashader.org/api.html#reductions), such as `ds.count()` or `ds.mean('val')`.
- A string (e.g. `'mean'`, `'count'`, `'min'`, `'max'`, etc.), in which case the aggregated dimension can be defined by setting the [`color`](option-color) option (if not, the first non-coordinate variable found is used).

The `'count_cat'` or `'by'` aggregators can be used for categorical cata. `ds.by(<column>, <reduction>)` allows to define the per-category reduction function (default is `count`). Alternatively, setting the [`by`](option-by) option to a categorical column is equivalent to setting `aggregator=ds.by(<cat_column>)`.

Two additional aggregators are available:
- `ds.summary(...)` can be used to compute multiple aggregates simultaneously, by defining a sequence of key/value pairs representing the aggregate labels and reductions. For example, setting `aggregator=ds.summary(min_s=ds.min('s'), max_s=ds.max('s'))` will make both the `min_s` and `max_s` aggregated values available in the hover tooltip.
- `ds.where(<selector_reduction>, lookup_column)` can be used to extract the values of another column based on a selector refuction (e.g. `'first'`, `'min'`). For example, setting `aggregator=ds.where(ds.min('s'), 'val')` will display in each pixel the value of the variable `'val'` where the variable `'s'` is minimum.

Let's start with the simple case, setting aggregator with a plain string and with a datashader reduction object.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata
import datashader as ds

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(x='x', y='y', data_aspect=1, frame_height=250)
df.hvplot.points(
    rasterize=True, aggregator='var', color='s',
    clabel='var(s)', **plot_opts
) +\
df.hvplot.points(
    rasterize=True, aggregator=ds.min('s'), clabel='min(s)',
    **plot_opts
)

The example below shows how `aggregator` can be used to handle categorical data.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata
import datashader as ds

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(x='x', y='y', data_aspect=1, frame_height=250)
df.hvplot.points(
    datashade=True, aggregator=ds.by('cat'),
    title="Categorical datashading with\n'count' aggregator'", **plot_opts
) +\
df.hvplot.points(
    datashade=True, aggregator=ds.by('cat', ds.min('s')), hover_cols=['s'],
    title="Categorical datashading with\n'min(s)' aggregator'", **plot_opts
)

The next examples show how to leverage `ds.summary()` and `ds.where()`. Hover over the plots to see how what information is made available in the tooltip.

In [ ]:
ds.summary(min_s=ds.min('s'), min_val=ds.min('val'))

In [ ]:
ds.where(ds.min('s'), 'val')

In [ ]:
ds.summary(min_s=ds.min('s'), min_val=ds.min('val'))

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata
import datashader as ds

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(x='x', y='y', data_aspect=1, frame_height=250)
df.hvplot.points(
    rasterize=True, title="summary(\n  min_s=min('s'), max_s=max('s')\n)",
    aggregator=ds.summary(min_s=ds.min('s'), min_val=ds.min('val')),
    **plot_opts
) +\
df.hvplot.points(
    rasterize=True, title="where(min('s', 'val')",
    aggregator=ds.where(ds.min('s'), 'val'),
    **plot_opts
)

(option-datashade)=
## `datashade`

The `datashade` option can be used to apply rasterization (aggregation into a grid of pixels) and colormapping operations using the [Datashader](https://datashader.org) library. Enabling this options allows:
- Rendering large datasets in the browser that would otherwise crash it as it can easily handle billions of data points.
- Dynamically exploring large datasets and discovering patterns, which would otherwise be difficult to find as plotting large data come with [many pitfalls](https://datashader.org/user_guide/Plotting_Pitfalls.html) such as overplotting.

This approach can turn even the largest datasets into an image that captures patterns such as density or value distribution, making it for example ideal for high-volume scatter plots. When `datashade=True`, hvPlot returns a [`DynamicMap`](inv:holoviews#reference/containers/bokeh/DynamicMap) containing an [`RGB`](inv:holoviews#reference/elements/bokeh/RGB) instead of individual glyphs.

:::{tip}
Since `datashade=True` produces an RGB image, the underlying data (e.g. the aggregated values per pixel) is not directly available to the plot. Enabling the `'hover'` [tool](option-tools) (disabled by default when `datashade=True`) would only show the RGB value per pixel, and no meaningful colorbar can be attached to the plot. To let the frontend apply colormapping instead of the backend, and as a consequence expose the underlying data, we recommend setting [`rasterize=True`](option-rasterize) instead of `datashade=True`.
:::

The [`cnorm`](option-cnorm) option defaults to `'eq_hist'` when `datashade=True`.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

df.hvplot.scatter(
    x='x', y='y', datashade=True, data_aspect=1, frame_height=250,
    title='Datashaded scatter plot with\n"count" aggregator and\n"eq_hist" cnorm'
)

In this example, the entire dataset is rasterized into a colormapped image, with denser areas appearing darker.

(option-downsample)=
## `downsample`

The `downsample` option can be used to dynamically reduce the number of plotted points by summarizing data before rendering, making it for example ideal for large timeseries datasets. This results in lighter plots and faster rendering.

Valid values include (most require the [`tsdownsample`](https://github.com/predict-idlab/tsdownsample) library to be installed):

- `False`: No downsampling is applied.
- `True`: Applies downsampling using HoloViews' default algorithm LTTB.
- `'lttb'`: Explicitly applies the [Largest Triangle Three Buckets algorithm (LTTB)](https://skemman.is/handle/1946/15343). Uses `tsdownsample` if installed, if not defers to HoloViews' LTTB implementation (slower).
- `'minmax'`: Applies the MinMax algorithm, selecting the minimum and maximum values in each bin. Requires ``tsdownsample``.
- `'m4'`: Applies the M4 algorithm, selecting the minimum, maximum, first, and last values in each bin. Requires ``tsdownsample``.
- `'minmax-lttb'`: Combines MinMax and LTTB algorithms for downsampling, first applying MinMax to reduce to a preliminary set of points, then LTTB for further reduction. Requires `tsdownsample`.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = (
    hvsampledata.stocks("pandas")
    .set_index("date")[["Apple"]]
    .resample("2Min")
    .interpolate(method="polynomial", order=5)
)
print(f"This dataset contains {len(df)} rows.")

df.hvplot.line(downsample="lttb", width=500, height=300, title="downsampled with lttb")

:::{note}
Requires `holoviews>=1.16`.
:::

(option-dynspread)=
## `dynspread`

When rendering with [`datashade=True`](option-datashade) or [`rasterize=True`](option-datashade), individual points can become hard to see, especially when sparse (1 isolated data point will color 1 pixel only which can be hard to see on a screen). Enabling `dynspread=True` dynamically increases the size of points in less dense areas, making them more visible.

In more details, spreading expands each pixel a certain number of pixels on all sides according to a circular shape, merging pixels using a compositing operator. Dynamic spreading determines how many pixels to spread based on a density heuristic. Spreading starts at 1 pixel, and stops when the fraction of adjacent non-empty pixels reaches the specified [`threshold`](option-threshold), or the [`max_px`](option-max_px) is reached, whichever comes first.

In the example below, we zoom in over an area with sparse data. The colored pixels are difficult to distinguish on the left plot, while they are clearly visible on the right plot with `dynspread=True`.

In [ ]:
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(
    x='x', y='y', frame_height=250, data_aspect=1,
    xlim=(-5.5, -5), ylim=(2.5, 3),
)
df.hvplot.scatter(
    rasterize=True, dynspread=False,
    title="Datashade without dynspread", **plot_opts,
) +\
df.hvplot.scatter(
    rasterize=True, dynspread=True,
    title="Datashade with dynspread", **plot_opts,
)

(option-max_px)=
## `max_px`

The `max_px` option sets the upper limit on how much [`dynspread`](option-dynspread) can increase point size (in pixels, default is `3`). It only applies when `dynspread=True`.

In [ ]:
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(
    x='x', y='y', frame_height=250, data_aspect=1,
    xlim=(-5.5, -5), ylim=(2.5, 3),
)
df.hvplot.scatter(
    rasterize=True, dynspread=True,
    title="Dynspread with max_px=3 (default)", **plot_opts,
) +\
df.hvplot.scatter(
    rasterize=True, dynspread=True, max_px=8,
    title="Dynspread with max_px=8", **plot_opts
)

:::{tip}
Larger values make sparse data more prominent but can also distort the visual scale if overused.
:::

(option-pixel_ratio)=
## `pixel_ratio`

This option adjusts the internal pixel resolution used by [`datashade`](option-datashade) or [`rasterize`](option-rasterize), relative to the actual display size.

:::{note}
By default, and if possible, `pixel_ratio` is inferred automatically from the browser and will be internally set to `2` on high-DPI screens to improve sharpness. You can override it manually for consistent rendering across devices.
:::

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

df.hvplot.scatter(
    x='x', y='y', datashade=True, pixel_ratio=0.1, frame_height=250,
    data_aspect=1, title="Datashade with low pixel ratio"
)

(option-precompute)=
## `precompute`

Operations that involve rasterization ([`rasterize`](option-rasterize) and [`datashade`](option-datashade)) can be computationally expensive as they operate on the full dataset (unlike e.g. [`dysnpread`](option-dyspread)). `precompute` can be set to `True` to get faster performance in interactive usage by caching the last set of data used in plotting (*after* any transformations needed) and reusing it when it is requested again. This is particularly useful when your data is not in one of the supported data formats already and needs to be converted. `precompute` is `False` by default, because it requires using memory to store the cached data, but if you have enough memory, you can enable it so that repeated interactions (such as zooming and panning) will be much faster than the first one. Learn more about this option in the [HoloViews large data user guide](https://holoviews.org/user_guide/Large_Data.html#cache-initial-processing-with-precompute-true).

:::{note}
In practice, most Datashader-plots don't need to do extensive precomputing, but enabling it for {meth}`hvplot.hvPlot.polygons` and {meth}`hvplot.hvPlot.quadmesh` plots can greatly speed up interactive usage.
:::

(option-rasterize)=
## `rasterize`

The `rasterize` option can be used to apply a rasterization (aggregation into a grid of pixels) operation using the [Datashader](https://datashader.org) library. Enabling this options allows:
- Rendering large datasets in the browser that would otherwise crash it as it can easily handle billions of data points.
- Dynamically exploring large datasets and discovering patterns, which would otherwise be difficult to find as plotting large data come with [many pitfalls](https://datashader.org/user_guide/Plotting_Pitfalls.html) such as overplotting.

This approach can turn even the largest datasets into an image that captures patterns such as density or value distribution, making it for example ideal for high-volume scatter plots. This option applies only rasterization, leaving colormapping to the plotting backend. Unlike [`datashade`](option-datashade), the returned [`DynamicMap`](inv:holoviews#reference/containers/bokeh/DynamicMap) does not contain an `RGB` element but data grid ([`Image`](inv:holoviews#reference/elements/bokeh/Image) or [`ImageStack`](inv:holoviews#reference/elements/bokeh/ImageStack)). This allows exposing the underlying data to the user interface via for example a colorbar and additional values displayed in the hover tooltip.

The [`cnorm`](option-cnorm) option defaults to `'linear'` when `datashade=True`.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

df.hvplot.scatter(
    x='x', y='y', rasterize=True, data_aspect=1, frame_height=250, cnorm='log',
    title='Rasterized scatter with count aggregator\nand log cnorm'
)

In this example, the entire dataset is rasterized into an image grid that is colormapped in the front-end, with denser areas appearing darker. Hover over the plot to see the count computed in each bin/pixel.

(option-resample_when)=
## `resample_when`

Operations like [`rasterize`](option-rasterize), [`datashade`](option-datashade), and [`downsample`](option-downsample) are very effective at displaying large datasets. When interacting with a plot, and for example zooming in over a region with a few points, these operations are in practice no longer needed. `resample_when` can be set to a number of data points present in the viewport below which resampling is toggled off. 

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

df.hvplot.scatter(
    x='x', y='y', rasterize=True, resample_when=1_000,
    data_aspect=1, frame_height=250, cnorm='log',
    title="Rasterize only when >1000 points in view"
)

When running the code above, you will notice that after zooming in enough, the original data points appear. This gives a hybrid experience: raw points at low density, rasterized aggregates when zoomed out.

(option-threshold)=
## `threshold`

Controls sensitivity for [`dynspread`](option-dynspread). A value of `1.0` always spreads sparse points, while `0.0` never does. Intermediate values let you tune spreading behavior. It only applies when `dynspread=True`.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

plot_opts = dict(
    x='x', y='y', datashade=True, dynspread=True,
    data_aspect=1, frame_width=200, xlim=(-2, 0), ylim=(7, 9),
)
df.hvplot.scatter(threshold=0.0, title="Dynspread threshold=0.0", **plot_opts) +\
df.hvplot.scatter(threshold=0.5, title="Dynspread threshold=0.5", **plot_opts) +\
df.hvplot.scatter(threshold=1.0, title="Dynspread threshold=1.0", **plot_opts)

(option-x_sampling__y_sampling)=
## `x_sampling` / `y_sampling`

Set minimum data resolution in the x and/or y direction when setting [`datashade`](option-datashade) or [`rasterize`](option-rasterize). This is useful to set the granularity of the pixel grid when zoomed in, or when visualizing images or gridded data that requires consistent resolution control.

In [ ]:
import hvplot.pandas  # noqa
import hvsampledata

df = hvsampledata.synthetic_clusters("pandas")

df.hvplot.scatter(
    x='x', y='y', rasterize=True, x_sampling=0.1, y_sampling=0.1,
    data_aspect=1, cnorm='log', xlim=(0, 1), ylim=(0, 1), frame_height=250,
    title='Zoomed in rasterized plot\nwith custom x/y-sampling'
)